Pràctica SPARK

Integrants:
    - David Morillo Massagué (1666540)
    - Albert Guillaumet Mata (1672344)
    - Adrià Muro Gómez (1665191)

In [ ]:
!sudo apt-get update
!sudo apt-get install openjdk-8-jre
!wget https://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
import findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better spark

In [ ]:
c = spark.read.csv(
    "customers.csv", header=True)

Use any of the previous transformations to find the answer to these questions. You can export your Jupyter notebook final version as part of the deliverable. For each question, you must provide the following information: • What command are you going to use? Why? • Which is your Spark operation to solve the question? • Which output is providing your Spark command (3 lines max.)

# Questions:

**1-How many elements can we find (in our DataFrame)?**

Volem saber quants registres hi ha en total al DataFrame, és a dir, quantes files representen compres o transaccions. Utilitzem count() perquè és una acció que retorna el nombre total de files del DataFrame, ideal per obtenir una visió general de la mida del dataset.





In [ ]:
c.count()

1003

Veiem que el Dataset té 1003 files

**2-How many unique customers?**

Volem saber quants clients diferents han fet compres, és a dir, quants valors únics hi ha a la columna customer. Utilitzem distinct() per eliminar duplicats de customer i després count() per obtenir el total de clients únics.

In [ ]:
c.select('customer').distinct().count()

31

Veiem que hi ha 31 clients únics

**3-How many products were purchased by each customer?**

Volem saber quants productes diferents ha comprat cada client al llarg de totes les seves compres. Amb groupBy() agrupem per client, i amb countDistinct() comptem els productes únics comprats per cada client.

In [ ]:
c.groupBy("customer").agg(expr("count(distinct(product)) as unique_products")).show(25)

+--------+---------------+
|customer|unique_products|
+--------+---------------+
|     125|             10|
|     124|             10|
|     101|             10|
|     112|             10|
|     113|             10|
|     110|             10|
|     107|             10|
|     100|             10|
|     126|             10|
|     120|             10|
|     130|             10|
|     118|             10|
|     104|             10|
|     128|             10|
|     102|             10|
|     111|             10|
|     103|             10|
|     115|             10|
|     122|             10|
|     108|             10|
|     117|             10|
|     114|             10|
|     106|             10|
|     116|             10|
|     105|             10|
+--------+---------------+
only showing top 25 rows



Veiem la cuantitat de productes comprats per a cada usuari

**4-Sort customers by quantity**

Ens interessa ordenar els clients segons la quantitat total de productes que han comprat (sumant totes les seves compres). Fem servir groupBy per client, sum("quantity") per sumar el total de productes comprats, i orderBy per ordenar de major a menor.

In [ ]:
c.groupBy("customer").agg(expr("sum(quantity) as total_quantity")).orderBy(desc("total_quantity")).show(5)

+--------+--------------+
|customer|total_quantity|
+--------+--------------+
|     101|         196.0|
|     122|         179.0|
|     117|         176.0|
|     100|         172.0|
|     124|         165.0|
+--------+--------------+
only showing top 5 rows



La taula mostra els id's dels primers 5 usuaris amb la quantitat comprada per cadascú.

**5-How many times customer id number 100 has purchased more than 5 items?**

Volem saber quantes vegades el client amb ID 100 ha fet una compra de més de 5 unitats. Utilitzem la comanda where() per seleccionar les files que compleixen les dues condicions.

In [ ]:
c.where((col("customer") == 100) & (col("quantity") > 5)).count()

16

El resultat 16 és la quantitat de vegades que el comprador (100) ha comprat més de 5 items.


**6-Which were the products bought by customer with the largest number of transactions? We are interested in the customer that has done more purchases. You do not need to consider quantities of products, just how many times a customer has done a transaction.**

Volem saber quins productes ha comprat el client que ha fet més transaccions (independentment de la quantitat comprada per transacció). Primer utilitzem groupBy i count() per saber quin client té més files (transaccions). Un cop identificat, utilitzem where() i distinct() per obtenir la llista de productes diferents que ha comprat.

In [ ]:
top_customer = c.groupBy("customer").count().orderBy(desc("count")).first()["customer"]
c.where(col("customer") == top_customer).select("product").distinct().show()

+-------+
|product|
+-------+
|      7|
|      3|
|      8|
|      5|
|      6|
|      9|
|      1|
|     10|
|      4|
|      2|
+-------+



 La taula mostra els diferents porductes de l'usuari que ha realitzat el màxim nombre de transaccions.